<a href="https://colab.research.google.com/github/emiliagyr/EstadisticaExperimental/blob/main/Colab/Ejercicio3_ANOVA_Solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pingouin
!pip install pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.7 MB/s eta 0:00:00


In [16]:
import pandas as pd
import seaborn as sns
import pingouin as pg

#Enunciado
La intervención Video-feedback Intervention to promote Positive Parenting and Sensitive Discipline (VIPP-SD) tiene como objetivo incrementar la sensibilidad materna mediante 7 sesiones en las que se filman y discuten distintas interacciones de la diada madre-niño. Debido a que esta intervención ha sido probada solamente en los Países Bajos, un grupo de investigación de la PUCP deseaba evaluar si se tendrían resultados positivos en una muestra piloto de madres peruanas. Para ello se trabajó con 120 diadas madre niño y se las evaluó mediante la escala de sensibilidad de Ainsworth en 4 momentos (antes de la intervención, después de la intervención, 6 meses después del final de la intervención y 12 meses después del final de la intervención).

a. El grupo de investigación buscaba identificar si habría diferencias en la sensibilidad materna en los distintos momentos a raíz de la intervención. Asuma normalidad en la distribución de las mediciones.

In [4]:
# VD: Sensibilidad materna
# VI: Intervención
# N1: Antes de la intervención
# N2: Después de la intervención
# N3: 6 meses dspués de la intervención
# N4: 12 meses después del final de la intervención

# Diseño: Intrasujeto

##Hipotesis
HO: N1 = N2 = N3 = N4

H1: Al menos 2 medidas son diferentes

#1. Leer los datos del documento .sav (SPSS) o .csv (Excel)

In [5]:
df = pd.read_spss("VIPP-SD.sav")
df

,Sujeto,NSE,Pre,Post,Post06m,Post12m
0,1.0,Medio,16.0,18.0,18.0,17.0
1,2.0,Medio,16.0,18.0,18.0,17.0
2,3.0,Bajo,12.0,14.0,14.0,13.0
3,4.0,Medio,15.0,18.0,18.0,17.0
4,5.0,Bajo,12.0,15.0,15.0,15.0
...,...,...,...,...,...,...
115,116.0,Bajo,12.0,15.0,15.0,14.0
116,117.0,Bajo,12.0,16.0,15.0,14.0
117,118.0,Bajo,13.0,16.0,15.0,14.0
118,119.0,Alto,13.0,16.0,16.0,15.0


In [12]:
melt_df = pd.melt(
    df, value_name="sensibilidad", var_name="Intervención", id_vars=["Sujeto", "NSE"]
)
melt_df

,Sujeto,NSE,Intervención,sensibilidad
0,1.0,Medio,Pre,16.0
1,2.0,Medio,Pre,16.0
2,3.0,Bajo,Pre,12.0
3,4.0,Medio,Pre,15.0
4,5.0,Bajo,Pre,12.0
...,...,...,...,...
475,116.0,Bajo,Post12m,14.0
476,117.0,Bajo,Post12m,14.0
477,118.0,Bajo,Post12m,14.0
478,119.0,Alto,Post12m,15.0


#2. Estadisticos descriptivos

In [13]:
df.describe()

,Sujeto,Pre,Post,Post06m,Post12m
count,120.000000,120.00000,120.000000,120.000000,120.000000
mean,60.500000,14.10000,16.908333,16.300000,15.658333
std,34.785054,2.06776,1.724738,2.124685,2.002082
min,1.000000,8.00000,12.000000,6.000000,10.000000
25%,30.750000,13.00000,16.000000,15.000000,14.000000
50%,60.500000,14.00000,17.000000,16.000000,16.000000
75%,90.250000,16.00000,18.000000,18.000000,17.000000
max,120.000000,18.00000,21.000000,20.000000,20.000000


# 3. Análisis de medias repetidas

Prueba de esfericidad de Mauchly

In [8]:
pg.sphericity(df)

SpherResults(spher=False, W=7.161204510064216e-10, chi2=2465.7949273216623, dof=14, pval=0.0)

No se asume esfericidad, por lo tanto se usa prueba de Greenhouse-Geisser

In [26]:
pg.rm_anova(
    data=melt_df,
    dv="sensibilidad",
    within="Intervención",
    subject="Sujeto",
    correction="GG",
)

,Source,ddof1,ddof2,F,p-unc,ng2,eps
0,Intervención,3,357,257.602983,6.125739e-89,0.218564,0.853338


In [23]:
pg.pairwise_tests(
    data=melt_df,
    dv="sensibilidad",
    within="Intervención",
    padjust="bonf",
    subject="Sujeto",
    effsize="cohen",
)

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,Intervención,Post,Post06m,True,True,5.501663,119.0,two-sided,2.190723e-07,1.314434e-06,bonf,5.518e+04,0.314372
1,Intervención,Post,Post12m,True,True,14.340333,119.0,two-sided,1.063063e-27,6.378376e-27,bonf,3.337e+24,0.668963
2,Intervención,Post,Pre,True,True,22.294143,119.0,two-sided,2.647226e-44,1.588336e-43,bonf,7.815e+40,1.474973
3,Intervención,Post06m,Post12m,True,True,7.761885,119.0,two-sided,3.220056e-12,1.932034e-11,bonf,2.447e+09,0.310841
4,Intervención,Post06m,Pre,True,True,18.263832,119.0,two-sided,2.571064e-36,1.542639e-35,bonf,1.021e+33,1.049410
5,Intervención,Post12m,Pre,True,True,14.929020,119.0,two-sided,4.832258e-29,2.899355e-28,bonf,6.976e+25,0.765696
